# Communities
Communities sind Gruppen oder Clustern von Knoten in einem Graphen, deren Verbindungen untereinander dichter sind als mit Knoten außerhalb der Gruppe. In einem sozialen Netzwerk könnte das beispielsweise bedeuten, dass eine Gruppe von Personen untereinander häufiger kommuniziert als mit Personen außerhalb der Gruppe. Die Erkennung der Communities kann somit strukturelle Muster oder soziale Gruppierungen innerhalb des Netzwerks enthüllen.

In [ ]:
import math
import random
from functools import reduce
from itertools import combinations

import numpy as np
import networkx as nx

np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

from tui_dsmt.graph import LabelPropagation, RandomWalk
from tui_dsmt.graph.datasets import load_community_tiny, load_community_small

tiny_community = load_community_tiny()
small_communities = load_community_small()

## Inhaltsverzeichnis
- [Clustering-Koeffizient](#Clustering-Koeffizient)
  - [Definition](#Definition)
  - [Schlichte Berechnung](#Schlichte-Berechnung)
  - [Schlichte Berechnung mit Adjazenzmatrix](#Schlichte-Berechnung-mit-Adjazenzmatrix)
  - [Berechnung mit Stichprobe](#Berechnung-mit-Stichprobe)
- [Communities](#Communities)
  - [Arten der Erkennung](#Arten-der-Erkennung)
  - [Label-Propagation-Algorithmus](#Label-Propagation-Algorithmus)
  - [Markov Cluster Algorithmus (MCL)](#Markov-Cluster-Algorithmus-MCL)

## Clustering-Koeffizient
### Definition
Der Clustering-Koeffizient ist eine Maßzahl, die angibt, wie stark die Nachbarschaft eines Knotens in einem Graphen untereinander verbunden ist. Der Clustering-Koeffizient eines Knotens wird typischerweise als Verhältnis der tatsächlichen Anzahl der Kanten zwischen den Nachbarknoten dieses Knotens zur maximal möglichen Anzahl von Kanten definiert. Ein hoher Clustering-Koeffizient deutet darauf hin, dass die Nachbarn eines Knotens stark miteinander verbunden sind, während ein niedrigerer Wert auf eine geringere lokale Verbundenheit hinweist.

Um den **Clustering-Koeffizienten eines einzelnen Knotens** zu berechnen, betrachten wir die Anzahl der möglichen Kanten zwischen den Nachbarknoten dieses Knotens und die tatsächliche Anzahl der vorhandenen Kanten zwischen diesen Nachbarn. Der Clustering-Koeffizient für einen Knoten $v_i$ wird mit einer Anzahl an Nachbarn $d_i$ und $k_i$ Kanten zwischen den Nachbarn wie folgt berechnet: $$ C_i = \begin{cases} \frac{k_i}{d_i * (d_i - 1) / 2} & \text{falls } d_i > 1 \\ 0 & \text{sonst} \end{cases} $$

Alternativ kann der Clustering-Koeffizient durch den Quotienten aus der Anzahl der Dreiecke, an denen der Knoten beteiligt ist, und der Anzahl der Tripel, in denen der Knoten der mittlere Knoten ist, bestimmt werden.

Ein Clustering-Koeffizient von $0$ bedeutet, dass keine der Nachbarknoten des betrachteten Knotens miteinander verbunden sind. Dies kann in einem Netzwerk auftreten, das vollständig zufällig oder stark zerstreut ist, ohne lokale Verbindungen zwischen den Knoten. Ein Clustering-Koeffizient von $1$ bedeutet dagegen, dass alle Nachbarknoten des betrachteten Knotens miteinander verbunden sind.

Der **globale Clustering-Koeffizient** bezeichnet dagegen den Durchschnitt der Clustering-Koeffizienten aller Knoten: $$ C = \sum_{i=1}^{n} \frac{C_i}{n} $$

Alternativ kann der Quotient aus der dreifachen Anzahl der Dreiecke im Graphen und der Anzahl der verbundenen Tripel im Graphen gebildet werden.

**Beispiel**:

In [ ]:
nx.draw(tiny_community, with_labels=True, font_color='whitesmoke')

Im vorangeganenen Graphen besitzen die Knoten also nun die folgenden Clustering-Koeffizienten:

| Knoten | $k_i$ | $d_i$ | $C_i$         |
| ------ | ----- | ----- | ------------- |
| $1$    | $1$   | $2$   | $1$           |
| $2$    | $1$   | $3$   | $\frac{1}{3}$ |
| $3$    | $0$   | $2$   | $0$           |
| $4$    | $0$   | $3$   | $0$           |
| $5$    | $1$   | $3$   | $\frac{1}{3}$ |
| $6$    | $0$   | $1$   | $0$           |

Der globale Clustering-Koeffizient liegt somit bei etwa $C = 0.27$ und ist als niedrig einzustufen.

### Schlichte Berechnung
Wie üblich lässt sich die Formel in Code übersetzen, wobei - ebenfalls wie üblich - das Finden der Lösung vergleichbar langsam erfolgt.

In [ ]:
C_i = {}

# über alle Knoten im Graphen iterieren
for node in tiny_community.nodes:
    # Nachbarn finden
    neighbors = list(tiny_community.neighbors(node))

    # Anzahl der Nachbarn speichern
    d_i = len(neighbors)

    # Anzahl der Kanten initialisieren
    k_i = 0

    # über alle Kombinationen benachbarter
    # Knoten iterieren
    for u, v in combinations(neighbors, 2):
        # falls Kante zwischen dem Paar
        # existiert, wird gezählt
        if tiny_community.has_edge(u, v):
            k_i += 1

    # Bestimmung von C_i nach Formel
    if d_i > 1:
        C_i[node] = k_i / (d_i * (d_i - 1) / 2)
    else:
        C_i[node] = 0

C_i

**Verständnisfrage:** Wie würde sich der Code verändern, wenn statt der Anzahl der Kanten unter den Nachbarn die Dreiecke gezählt würden?

In [ ]:
C = sum(C_i.values()) / len(C_i)
C

### Schlichte Berechnung mit Adjazenzmatrix
Da jeder Graph eindeutig durch seine Adjazenzmatrix beschreibbar ist, eignet auch diese sich zur Berechnung des Clustering-Koeffizienten. Die Matrix für den Graphen `tiny_community` sieht wie folgt aus:

In [ ]:
A = nx.to_numpy_array(tiny_community)
A

Sei nun $k_i$ die Anzahl der $1$ in der Reihe $i$ bzw. $k_i = \sum_j A_{ij}$.

Dann lässt sich der globale Clustering-Koeffizient ausdrücken als:
$$
C = \frac{\sum_{ijk} A_{ij} A_{jk} A_{ki}}{\sum_i k_i * (k_i - 1)}
$$

In [ ]:
numerator = 0.
for i in range(len(A)):
    for j in range(len(A)):
        for k in range(len(A)):
            numerator += A[i,j] * A[j,k] * A[k,i]

denominator = 0.
for i in range(len(A)):
    denominator += sum(A[i]) * (sum(A[i]) - 1)

numerator / denominator

Auch für diesen Wert müssen aber in großen Graphen zahlreiche Matrizenmultiplikationen durchgeführt werden.

### Berechnung mit Stichprobe
Alternativ lassen sich approximative Verfahren anwenden, die basierend auf einer Stichprobe den Wert ermitteln.

In [ ]:
C_i = {}

# über eine zufällige Stichprobe aller
# Knoten im Graphen iterieren
for node in random.choices(tuple(tiny_community.nodes), k=3):
    # Nachbarn finden
    neighbors = list(tiny_community.neighbors(node))

    # Anzahl der Nachbarn speichern
    d_i = len(neighbors)

    # Anzahl der Kanten initialisieren
    k_i = 0

    # über alle Kombinationen benachbarter
    # Knoten iterieren
    for u, v in combinations(neighbors, 2):
        # falls Kante zwischen dem Paar
        # existiert, wird gezählt
        if tiny_community.has_edge(u, v):
            k_i += 1

    # Bestimmung von C_i nach Formel
    if d_i > 1:
        C_i[node] = k_i / (d_i * (d_i - 1) / 2)
    else:
        C_i[node] = 0

C = sum(C_i.values()) / len(C_i)
C

In einem solch kleinen Graphen schwankt die ermittelte Größe natürlich stark in Abhängigkeit der gewählten Stichprobengröße. In größeren Graphen ist dagegen mit einem besseren Ergebnis zu rechnen, während der Aufwand der Berechnung stark reduziert wird.

## Communities
Communities sind eine Teilmenge von Knoten in einem Graphen, die untereinander stärker verbunden sind als mit Knoten außerhalb ihrer eigenen Community. Informell betrachtet könnte in einem Netzwerk aus Nutzern eines Messenger-Dienstes beispielsweise eine Community aus Menschen bestehen, die untereinander häufiger Textnachrichten austauschen als mit Menschen außerhalb ihrer Community.

Die Erkennung von Communities teilt sich dabei in zwei unterschiedliche Bereiche.

### Arten der Erkennung
Der **memberbasierte** Ansatz zur Identifizierung von Communities konzentriert sich auf die Eigenschaften einzelner Knoten, um darauf basierend die Zugehörigkeit zu diesen Communities festzustellen. Dafür wird jeder Knoten einzeln betrachtet und anschließend bestimmt, welcher Community er am wahrscheinlichsten angehört.

Der **gruppenbasierte** Ansatz betrachtet die Beziehungen und Verbindungen innerhalb einer Gruppe als gemeinsame Eigenschaft, um Communitites zu erkennen.

Die Einordnung ist nicht immer trennscharf. So weist der nachfolgende Algorithmus zwar den Knoten eine Community zu, indem ausschließlich die lokale Nachbarschaft betrachtet wird, über die Iterationen hinweg verbreitet sich diese Information jedoch in Abhängigkeit der Verbindungen untereinander potentiell über den gesamten Graphen.

### Label-Propagation-Algorithmus
Der Label Propagation Algorithmus ist ein einfacher Algorithmus zur Erkennung von Communities in einem ungerichteten Graphen. Er basiert auf der Idee, dass Informationen (Labels) über die Zugehörigkeit zu Clustern oder Gemeinschaften innerhalb des Graphen von Knoten zu Knoten propagiert werden. Der Algorithmus ist besonders gut geeignet für große Graphen, in denen die Struktur der Gemeinschaften nicht stark ausgeprägt ist.

Der Algorithmus durchläuft folgende Schritte:
1. **Initialisierung**: Jeder Knoten im Graph wird zunächst mit einem eindeutigen Label versehen. (Bei der Verwendung von NetworkX sind Knotennamen eindeutig.)
2. **Label-Propagation:** In jeder Iteration des Algorithmus wird für jeden Knoten das Label basierend auf den Labels seiner Nachbarn aktualisiert. Dazu wird ein Mehrheitssystem eingesetzt, sodass ein Knoten das Label übernimmt, das innerhalb seiner Nachbarschaft am häufigsten vorkommt. (Falls mehrere Labels gleich häufig vorkommen, wird eines zufällig ausgewählt.)
3. **Einteilung:** Nach erreichen eines stabilen Zustands, werden die Knoten entsprechend ihrer Labels in Communities aufgeteilt. Knoten mit dem selben Label gehören zur selben Community.

Das Ergebnis ist von der Reihenfolge der Iteration über die Knoten abhängig. Eine zufällige Reihenfolge zu wählen verhindert, dass der Algorithmus in lokalen Minima oder Maxima steckenbleibt.

Als Beispiel verwenden wir einen Graphen, der etwas größer ist und offensichtlich drei Communities enthält.

In [ ]:
nx.draw(small_communities, with_labels=True, font_color='whitesmoke')

Mit NetworkX sieht der Algorithmus dann wie folgt aus:

In [ ]:
# Das Label aller Knoten wird auf den eindeutigen
# Namen des Knotens gesetzt.
for node in small_communities:
    small_communities.nodes[node]['label'] = node

# Die Variable steuert die Schleife. Die Schleife
# soll so lang laufen, wie noch Änderungen an
# den Labels gemacht werden.
changes = True

while changes:
    # Die Variable wird auf False gesetzt, sodass
    # nach dem Schleifendurchlauf abgebrochen wird,
    # sofern keine Änderungen gemacht werden.
    changes = False

    # Jeder Knoten im Graphen wird in zufälliger
    # Reihenfolge betrachtet.
    nodes = list(small_communities.nodes)
    random.shuffle(nodes)

    for node in nodes:
        # Die Labels aller Nachbarn werden zusammen
        # mit ihrer Häufigkeit gespeichert.
        neighbor_labels = {}

        for neighbor in small_communities.neighbors(node):
            n_label = small_communities.nodes[neighbor]['label']
            neighbor_labels[n_label] = neighbor_labels.get(n_label, 0) + 1

        # Eines (!) der häufigsten Label unter
        # den Nachbarn wird herausgesucht.
        mc_labels_max = max(neighbor_labels.values())
        mc_labels = [nl for nl in neighbor_labels if neighbor_labels[nl] == mc_labels_max]
        mc_label = random.choice(mc_labels)

        # Falls dem betrachteten Knoten nicht bereits
        # dieses Label zugeordnet wurde, wird es
        # aktualisiert und der nächste Schleifen-
        # durchlauf durch Setzen der Variable
        # erlaubt.
        if mc_label != small_communities.nodes[node]['label']:
            small_communities.nodes[node]['label'] = mc_label
            changes = True

# Zuletzt werden die Knoten nach dem zugeordneten
# Label gruppiert.
communities = {}

for node in small_communities:
    label = small_communities.nodes[node]['label']
    communities[label] = communities.get(label, []) + [node]

communities

Sie können dem Algorithmus auch schrittweise folgen. Wie bereits erwähnt wurde, kann das mehrfache Ausführen durch den Zufallsanteil zu unterschiedlichen Ergebnissen führen.

In [ ]:
LabelPropagation(small_communities)

### Markov Cluster Algorithmus (MCL)
Der [Markov Cluster Algorithmus](https://doi.org/10.1137/040608635) ist ein weiterer Algorithmus zur Community-Erkennung in Graphen. Er basiert auf dem Konzept der Markov-Ketten und simuliert den Prozess der Ausbreitung von Informationen in einem Graphen. Eine Markov-Kette ist dabei ein stochastisches System, dessen nächster Zustand ausschließlich vom aktuellen Zustand und den gegebenen Übergangswahrscheinlichkeiten abhängt.

Die zugrundeliegende Idee für den Algorithmus sind sogenannte *Random Walks*, bei denen ausgehend von einem der Knoten eine zufällige Kantenfolge gewählt und abgeschritten wird. Die ausgehenden Kanten eines Knotens markieren die von dort erreichbaren Zustände bzw. Knoten - es können also innerhalb eines Schrittes nur direkte Nachbarn erreicht werden. Dabei ergeben sich automatisch Muster:
- Ist der Graph in mehrere Komponenten zerfallen, werden nur Knoten innerhalb der Komponente erreicht, in der gestartet wird.
- Die Wahrscheinlichkeit innerhalb einer Community zu bleiben ist höher als sie zu verlassen.
- Knoten, die eine verbindende Funktion aufweisen, werden häufiger durchschritten als Knoten, die "Sackgassen" darstellen.

Die nachfolgende Zelle zeigt Ihnen einen Random Walk im bereits zuvor gezeigten Graphen. Mit dem Parameter `steps` können Sie die Anzahl der Schritte verändern. Führen Sie die Zelle mehrfach aus und beobachten Sie den Verlauf, da jedes Mal ein zufälliger Startknoten gewählt wird.

In [ ]:
RandomWalk(small_communities, steps=80)

Als *Markov-Ketten-Matrix* wird die Adjazenzmatrix des Graphen verwendet. Um als Wahrscheinlichkeitsverteilung des Übergangs von einem Knoten zum nächsten zu fungieren, muss diese jedoch spaltenweise normalisiert werden.

In [ ]:
def normalize(M):
    return M / M.sum(axis=0)[np.newaxis, :]

In [ ]:
adj = nx.to_numpy_array(small_communities)
n_adj = normalize(adj)

np.round(n_adj, 2)

Der Random Walk wird dann simuliert durch zwei abwechselnde Operationen:
- Die *Expansion* sorgt für die Ausbreitung der Wahrscheinlichkeiten innerhalb des Graphen. Der Schritt wird durchgeführt, indem die Markov-Ketten-Matrix mit sich selbst multipliziert wird. (Dies kann auch mit einer höheren Potenz als $2$ geschehen!)
- Die *Inflation* stärkt stark verbundene Knoten und schwächt schwach verbundene Knoten weiter ab. Dazu werden die Werte der Markov-Ketten-Matrix elementweise potenziert und anschließend (erneut) normalisiert. Die Wahl der Potenz hat starken Einfluss auf die Granularität der entstehenden Communities.

In [ ]:
def expand(M, e=2):
    return reduce(np.dot, (M,) * e)

def inflate(M, r):
    return normalize(M ** r)

Der Algorithmus führt die beiden Operationen nun ausgehend von der normalisierten Adjazenzmatrix aus und wiederholt die Schritte so lang, bis das Ergebnis konvergiert:

In [ ]:
e = 3
r = 3

M = n_adj
last_M = np.zeros(M.shape)

while abs(M - last_M).sum() / math.prod(M.shape) > 1e-3:
    last_M = M

    M = expand(M, e)
    M = inflate(M, r)

np.round(M, 3)

In der Regel konvertieren die Matrizen entgegen einer dünn besetzten Matrix, in der pro Zeile alle Werte, die nicht $0$ sind, ungefähr die gleiche Größe besitzen. Zuletzt muss die entstandene Matrix allerdings noch interpretiert werden. Dazu werden die Knoten in sogenannte anziehende und normale Knoten unterteilt, wobei erstere mindestens einen positiven Wert in ihrer Reihe besitzen.

Anziehende Knoten werden mit anderen zu einer Community zusammengefasst, wenn zwischen ihnen laut der konvergierten Matrix ein Anziehungsverhältnis besteht.

In [ ]:
communities = [set((row,)) for row in range(M.shape[0])]

for row in range(M.shape[0]):
    for col in range(M.shape[1]):
        if M[row, col] > 1e-9:
            communities[row].update(communities[col])
            communities[col] = communities[row]

set(map(frozenset, communities))